# ⚙️ 1. SZEKCIÓ – CSV → PostgreSQL előkészítés

### ✅ Hozz létre egy PostgreSQL adatbázist ecommerce néven.

In [1]:
import pandas as pd

# csak az első 100 sort olvassuk be
df_preview = pd.read_csv("ECommerce_consumer_behaviour.csv", nrows=100, encoding='latin1', on_bad_lines='skip')
df_preview.head(20)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
5,2425083,49125,1,2,18,NaN,91,6,0,16,dairy eggs,soy lactosefree
6,2425083,49125,1,2,18,NaN,120,7,0,16,dairy eggs,yogurt
7,2425083,49125,1,2,18,NaN,59,8,0,15,canned goods,canned meals beans
8,2425083,49125,1,2,18,NaN,35,9,0,12,meat seafood,poultry counter
9,1944304,162867,1,3,17,NaN,37,1,0,1,frozen,ice cream ice


### ✅ DbGate - Create database

### ✅ DbGate - Create table

### ✅ DbGate - Copy data into table

### ✅ Írj Python kódot, ami megszámolja, hány sor került be az adatbázisba, és összehasonlítja a CSV sorainak számával → ha eltérés van, jelezzen.

In [5]:
import yaml
import psycopg2
import pandas as pd

# Reading yaml file
with open("/home/gdaniel1979/auth/ecommerce.yaml", "r") as f:
    config = yaml.safe_load(f)

db_config = config["database"]

# Connecting to database
conn = psycopg2.connect(
    host = db_config["host"],
    port = db_config["port"],
    dbname = db_config["dbname"],
    user = db_config["user"],
    password = db_config["password"]
)

# Database count
query = "SELECT COUNT(*) FROM consumer_behavior;"
database_count = pd.read_sql(query, conn)
conn.close()

# CSV count
file_path = "/home/gdaniel1979/hobby_projects/consumer_behavior/ECommerce_consumer_behaviour.csv"
chunksize = 100000

row_count = 0
for chunk in pd.read_csv(file_path, chunksize = chunksize):
    row_count += len(chunk)

database_count = database_count.iloc[0, 0] # Extrating value of the first column, and the first row
print(f"Row count in database: {database_count}")
print(f"Row count in csv: {row_count}")
print(f"Difference: {database_count - row_count}")

/tmp/ipykernel_2874672/3775710739.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  database_count = pd.read_sql(query, conn)


Row count in database: 2019501
Row count in csv: 2019501
Difference: 0


### ✅ Készíts logolást: ha egy sor hibás és nem tölthető be, írd ki külön logfájlba.
#### Mivel már a tábla fel van töltve adatokkal, ezért törölni kell a tartalmát
#### LOG fájl elejére fűzi mindig az új információt (prepend)

In [6]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text 
import yaml
import time
from datetime import datetime
import os
import traceback

# === Settings ===
csv_path = "/home/gdaniel1979/hobby_projects/consumer_behavior/ECommerce_consumer_behaviour.csv"
yaml_path = "/home/gdaniel1979/auth/ecommerce_truncate.yaml"
log_path = "/home/gdaniel1979/hobby_projects/consumer_behavior/load_log.txt"

# === Reading password from yaml file ===
with open(yaml_path, "r") as f:
    creds = yaml.safe_load(f)

# === Connecting to database with SQLAlchemy engine ===
engine = create_engine(
    f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['database']}"
)

# === Main process ===
start_time = time.time()
total_loaded = 0
total_errors = 0
error_chunks = []

# === Truncate table before load ===
table_name = "consumer_behavior"

with engine.begin() as conn:
    conn.execute(text(f"TRUNCATE TABLE {table_name};"))

# === Load in chunks ===
chunk_size = 50000

for chunk_number, chunk in enumerate(pd.read_csv(csv_path, chunksize = chunk_size), start = 1):
    try:
        chunk.to_sql(table_name, engine, if_exists = "append", index = False)
        total_loaded += len(chunk)
    except Exception as e:
        total_errors += len(chunk)
        error_chunks.append((chunk_number, str(e)))
        traceback.print_exc()

# === Summary info ===
elapsed = time.time() - start_time
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

new_log = []
new_log.append("=" * 70)
new_log.append(f"Run summary: {timestamp}")
new_log.append("=" * 70)
new_log.append(f"Table {table_name} loaded.")
new_log.append(f"{total_loaded} rows loaded successfully.")
new_log.append(f"Loading of {total_errors} rows failed.")
new_log.append(f"Elapsed time: {elapsed:.2f} seconds.")

if error_chunks:
    new_log.append("Failed chunks:")
    for num, msg in error_chunks:
        new_log.append(f"   - Chunk {num}: {msg[:120]}") # only the first 120 characters
else:
    new_log.append("Every chunk successfully loaded. ✅")

new_log.append("\n")

# === Prepend log ===
if os.path.exists(log_path):
    with open(log_path, "r", encoding = "utf-8") as f:
        old_content = f.read()
else:
    old_content = ""

final_log = "\n".join(new_log) + "\n" + old_content

with open(log_path, "w", encoding = "utf-8") as f:
    f.write(final_log)

# === Close engine ===
engine.dispose()
print("Loading finished. Details in log file.")

Loading finished. Details in log file.


# 🟦 2. SZEKCIÓ – SQL lekérdezések PostgreSQL-ben

## Listázd ki az összes oszlopot LIMIT 10-zel.

## Számold meg hány egyedi user_id van.

## Melyik department_id-ből van a legtöbb termék?

## Listázd ki, mely product_name szerepel a legtöbbször.

## Számold ki, hány új rendelés (reordered = 0) és hány ismételt rendelés (reordered = 1) van.

## Melyik order_hour_of_day érték a leggyakoribb?

## Csoportosíts order_dow szerint és számold meg, melyik napon rendelnek a legtöbbet.

## 🟢 Alap szint – Lekérdezés, szűrés, rendezés (1–10)
## Listázd ki az összes sort, ahol department = 'produce'

## Listázd azokat a sorokat, ahol a reordered = 1 és a department = 'dairy eggs'

## Rendezd a vásárlásokat order_hour_of_day szerint növekvő sorrendbe

## Mutasd meg azokat a sorokat, ahol a days_since_prior_order érték nem NULL.

## Szűrd le azokat a vásárlásokat, ahol a order_dow értéke 0 vagy 6 (hétvége).

## Listázd az első 5 sort, ahol a add_to_cart_order értéke nagyobb, mint 10.

## Mutasd meg, melyik department alatt vannak a legtöbb product_name ismétlődések.

## Számold meg, hány sorban van days_since_prior_order értéke NULL.

## Melyik 5 department szerepel a legtöbbet a táblában?

## Hány különböző department szerepel az adatokban?

## 🟡 Középszint – Csoportosítás, aggregálás (11–25)
## Számold meg, hány rendelés (order_id) tartalmazott több mint 10 terméket.

## Számold meg, hány rendeléshez tartozik legalább 1 újrarendelt (reordered = 1) termék.

## Számold ki, hány termék szerepel átlagosan egy order_id alatt.

## Számold ki, hány különböző product_id tartozik az egyes department_id-khoz.

## Hány különböző product_name-et rendeltek az adott user_id felhasználók összesen? (Felhasználónként azt szeretné tudni a kérdés, hogy összesen hány különböző terméket rendelt élete során (az összes rendelését figyelembe véve)

## Hány különböző nap (order_dow) alatt rendeltek egy-egy user_id felhasználók?

## Csoportosíts order_dow szerint, és add meg az átlagos order_hour_of_day értéket naponta.

## Számold meg, hány új terméket (reordered = 0) rendeltek az egyes departmentekben.

## Számold meg, hány újrarendelt termék (reordered = 1) van departmentenként.

## Készíts kimutatást: departmentenként a rendelések számát és a különböző product_id-k számát.

## Melyik user_id adta le a legtöbb rendelést?

## Melyik user_id vásárolta a legtöbb különböző product_id-t?

## Hány user_id vásárolt legalább 3 különböző departmentből?

## Számold meg, hány rendelésben (order_id) van több, mint egy azonos termék (product_id).

## 🟠 Haladóbb – Elemzés és arányok (26–40)
## Számold ki, hogy az összes rendelés közül hány százaléknyi rendelés tartalmaz legalább egy újrarendelt (reordered = 1) terméket.

## Hasonlítsd össze a reordered = 0 és reordered = 1 arányát departmentenként.

## Számold meg, hány felhasználónak volt legalább egy ismételt rendelése.

## Mennyi az átlagos days_since_prior_order érték azoknál a rendeléseknél, ahol van újrarendelés?

## Készíts kimutatást, melyik napszakban (order_hour_of_day) a legnagyobb az újrarendelési arány.

## Csoportosíts department szerint, és számold ki, hány százalékát teszik ki az újrarendelések

## Melyik 3 department-ben a legmagasabb az újrarendelési arány?

## Melyik 3 department-ben a legalacsonyabb az újrarendelési arány?

## A „produce” (gyümölcsök/zöldségek) department rendeléseit vizsgálva melyik hét napján (order_dow) történik a legtöbb rendelés?

## Melyik órában (order_hour_of_day) vásárolják leggyakrabban a dairy eggs termékeket?

## Hány felhasználónak van legalább 10 rendelése?

## Hány olyan felhasználó van, aki kizárólag produce departmentből vásárolt?

## Melyik departmentben a legnagyobb az átfedés (ugyanaz a user több departmentből vásárol)?

## Számold ki, az add_to_cart_order átlagos értéke hogyan változik napszakonként.

## 🔵 Komplex – Több táblás logika helyett összetettebb egytáblás feladatok (41–50)
## Azonosítsd, mely termékek (product_name) kerülnek leggyakrabban a kosár első helyére (add_to_cart_order = 1).

## Azonosítsd, mely termékek a leggyakrabban utolsók a kosárban (add_to_cart_order legnagyobb érték per order_id).

## Számold meg, hány egyedi order_id tartalmaz duplikált product_id értékeket.

## Számold ki, hogy az egyes department kategóriákban átlagosan hány termék kerül a kosárba.

## Készíts rangsort: mely departmentekben a leggyakoribb, hogy a days_since_prior_order > 20.

## Számold meg, hogy egy rendelésben átlagosan hány különböző department szerepel.

## Készíts kimutatást, hogy hány különböző department van felhasználónként, és mi az átlaguk.

## Számold ki az újrarendelési arányt (reordered = 1 arány) user_id szerint, majd sorold a top 10 leghűségesebb vásárlót.

# 🔁 3. SZEKCIÓ – Python alapgyakorlatok (list, dict, string)

## Olvasd be a CSV első 50 sorát.

In [1]:
import pandas as pd

df = pd.read_csv("ECommerce_consumer_behaviour.csv", encoding='utf-8', sep=',', nrows=5000)
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


## Írj ciklust: számold meg, hány sor tartalmaz fresh fruits.

In [32]:
print(df.columns)

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'product_id', 'add_to_cart_order',
       'reordered', 'department_id', 'department', 'product_name'],
      dtype='object')


In [33]:
count = 0
product = 'fresh fruits'
for value in df['product_name']:
    if value == product:
        count += 1
count

606

## Hozz létre egy set-et az összes department névből.

In [34]:
departments = set(df['department'].dropna())
departments

{'alcohol',
 'babies',
 'bakery',
 'beverages',
 'breakfast',
 'bulk',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'international',
 'meat seafood',
 'missing',
 'other',
 'pantry',
 'personal care',
 'pets',
 'produce',
 'snacks'}

## Készíts egy dictionary-t, amely azt tárolja, hogy hány termék van departmentenként.

In [35]:
df['department'].value_counts().to_dict()

{'produce': 1506,
 'dairy eggs': 858,
 'snacks': 446,
 'beverages': 413,
 'frozen': 330,
 'pantry': 254,
 'bakery': 192,
 'canned goods': 160,
 'deli': 156,
 'dry goods pasta': 129,
 'household': 112,
 'meat seafood': 106,
 'breakfast': 105,
 'personal care': 72,
 'babies': 71,
 'international': 35,
 'alcohol': 20,
 'pets': 12,
 'missing': 11,
 'bulk': 8,
 'other': 4}

In [36]:
department_counts = {}
for dept in df['department']:
    if pd.notna(dept):
        if dept not in department_counts:
            department_counts[dept] = 1
        else:
            department_counts[dept] += 1
sorted(department_counts.items(), key=lambda x: x[1], reverse=True)

[('produce', 1506),
 ('dairy eggs', 858),
 ('snacks', 446),
 ('beverages', 413),
 ('frozen', 330),
 ('pantry', 254),
 ('bakery', 192),
 ('canned goods', 160),
 ('deli', 156),
 ('dry goods pasta', 129),
 ('household', 112),
 ('meat seafood', 106),
 ('breakfast', 105),
 ('personal care', 72),
 ('babies', 71),
 ('international', 35),
 ('alcohol', 20),
 ('pets', 12),
 ('missing', 11),
 ('bulk', 8),
 ('other', 4)]

## Rendezd a dept → count dictionary-t érték szerint csökkenően.

In [37]:
df['department'].value_counts().to_dict()

{'produce': 1506,
 'dairy eggs': 858,
 'snacks': 446,
 'beverages': 413,
 'frozen': 330,
 'pantry': 254,
 'bakery': 192,
 'canned goods': 160,
 'deli': 156,
 'dry goods pasta': 129,
 'household': 112,
 'meat seafood': 106,
 'breakfast': 105,
 'personal care': 72,
 'babies': 71,
 'international': 35,
 'alcohol': 20,
 'pets': 12,
 'missing': 11,
 'bulk': 8,
 'other': 4}

## Szűrd ki azokat a sorokat, ahol reordered == 1, és tedd listába.

In [44]:
filtered_df = df[df['reordered'] == 1]
rows_as_list = filtered_df['department'].values.tolist()
rows_as_list

['produce',
 'produce',
 'dairy eggs',
 'dairy eggs',
 'dairy eggs',
 'dairy eggs',
 'dairy eggs',
 'produce',
 'dairy eggs',
 'produce',
 'dairy eggs',
 'produce',
 'produce',
 'bakery',
 'dairy eggs',
 'produce',
 'produce',
 'produce',
 'produce',
 'produce',
 'dairy eggs',
 'dairy eggs',
 'beverages',
 'pantry',
 'beverages',
 'dairy eggs',
 'beverages',
 'produce',
 'snacks',
 'household',
 'household',
 'dairy eggs',
 'personal care',
 'babies',
 'produce',
 'snacks',
 'personal care',
 'produce',
 'produce',
 'produce',
 'dairy eggs',
 'produce',
 'meat seafood',
 'deli',
 'produce',
 'snacks',
 'beverages',
 'produce',
 'meat seafood',
 'produce',
 'meat seafood',
 'meat seafood',
 'produce',
 'deli',
 'bakery',
 'pantry',
 'dairy eggs',
 'frozen',
 'dairy eggs',
 'dairy eggs',
 'produce',
 'bakery',
 'snacks',
 'beverages',
 'canned goods',
 'deli',
 'pantry',
 'household',
 'produce',
 'snacks',
 'canned goods',
 'dry goods pasta',
 'dairy eggs',
 'dairy eggs',
 'dry goods pa

## Írj kódot, amely megnézi, van-e duplikált product_id a mintában.

In [5]:
df['product_id'].duplicated()

0       False
1       False
2       False
3       False
4        True
        ...  
4995     True
4996     True
4997     True
4998     True
4999     True
Name: product_id, Length: 5000, dtype: bool

In [9]:
duplicates = df[df['product_id'].duplicated()]
duplicates

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
5,2425083,49125,1,2,18,NaN,91,6,0,16,dairy eggs,soy lactosefree
11,1944304,162867,1,3,17,NaN,83,3,0,4,produce,fresh vegetables
13,1944304,162867,1,3,17,NaN,91,5,0,16,dairy eggs,soy lactosefree
14,1944304,162867,1,3,17,NaN,24,6,0,4,produce,fresh fruits
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2031129,152365,14,1,16,2.0,78,1,1,19,snacks,crackers
4996,2031129,152365,14,1,16,2.0,34,2,0,1,frozen,frozen meat seafood
4997,2031129,152365,14,1,16,2.0,83,3,0,4,produce,fresh vegetables
4998,2031129,152365,14,1,16,2.0,26,4,1,7,beverages,coffee


In [12]:
duplicated_ids = df['product_id'][df['product_id'].duplicated()].unique()
sorted(duplicated_ids)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 134]

In [13]:
has_duplicates = df['product_id'].duplicated().any()
has_duplicates

True

# 🟨 4. SZEKCIÓ – Python + PostgreSQL (psycopg2 SELECT és feldolgozás)

## Írj Python függvényt, ami lekéri az összes produce department rekordot adatbázisból.

In [4]:
def get_produce_records(df):
    result = df[df['department'] == 'produce']
    return result

get_produce_records(df).head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
10,1944304,162867,1,3,17,NaN,24,2,0,4,produce,fresh fruits
11,1944304,162867,1,3,17,NaN,83,3,0,4,produce,fresh vegetables
14,1944304,162867,1,3,17,NaN,24,6,0,4,produce,fresh fruits


In [1]:
import psycopg2
import yaml

def get_produce_records_sql(yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"):
    try:
        # yaml beolvasás
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # Lekérdezés
        query = "SELECT * FROM consumer_behavior WHERE department = %s LIMIT 1000;"
        cursor.execute(query, ('produce',))
        records = cursor.fetchall()
        cursor.close()
        conn.close()

        return records

    except Exception as e:
        print("Hiba történt:", e)
        return None

records = get_produce_records_sql("/home/gdaniel1979/auth/ecommerce.yaml")
records

[(3311293, 70116, 5, 5, 8, 4.0, 24, 2, 1, 4, 'produce', 'fresh fruits'),
 (3311293,
  70116,
  5,
  5,
  8,
  4.0,
  123,
  4,
  1,
  4,
  'produce',
  'packaged vegetables fruits'),
 (3311293,
  70116,
  5,
  5,
  8,
  4.0,
  123,
  5,
  1,
  4,
  'produce',
  'packaged vegetables fruits'),
 (3311293,
  70116,
  5,
  5,
  8,
  4.0,
  123,
  6,
  1,
  4,
  'produce',
  'packaged vegetables fruits'),
 (3311293, 70116, 5, 5, 8, 4.0, 83, 7, 1, 4, 'produce', 'fresh vegetables'),
 (3311293, 70116, 5, 5, 8, 4.0, 83, 8, 0, 4, 'produce', 'fresh vegetables'),
 (3311293, 70116, 5, 5, 8, 4.0, 83, 9, 0, 4, 'produce', 'fresh vegetables'),
 (853432,
  126800,
  9,
  5,
  11,
  30.0,
  123,
  1,
  1,
  4,
  'produce',
  'packaged vegetables fruits'),
 (853432, 126800, 9, 5, 11, 30.0, 24, 2, 1, 4, 'produce', 'fresh fruits'),
 (853432, 126800, 9, 5, 11, 30.0, 24, 3, 1, 4, 'produce', 'fresh fruits'),
 (853432,
  126800,
  9,
  5,
  11,
  30.0,
  123,
  5,
  1,
  4,
  'produce',
  'packaged vegetables fr

## Írj függvényt, ami visszaadja TOP 5 legtöbbet rendelt product_id-t (csak SQL).

In [6]:
import psycopg2
import yaml
import pandas as pd

def get_top5_productid_sql():
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # Lekérdezés
        query = """
        SELECT product_id, COUNT(product_id) AS cnt 
        FROM consumer_behavior 
        GROUP BY product_id 
        ORDER BY cnt DESC LIMIT 5;
        """
        df = pd.read_sql_query(query, conn)
        conn.close()

        return df

    except Exception as e:
        print("Hiba történt:", e)
        return None

records = get_top5_productid_sql()
records

/tmp/ipykernel_3177636/3702174990.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,product_id,cnt
0,24,226039
1,83,212611
2,123,109596
3,120,90751
4,21,61502


## Írj függvényt, ami user_id alapján szűr, és listát ad vissza rendelésekről.

In [10]:
import psycopg2
import psycopg2.extras
import yaml

def get_userid_sql():
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

        # Lekérdezés
        query = """
        SELECT * 
        FROM consumer_behavior 
        WHERE user_id < 10000 
        LIMIT 5;
        """
        cursor.execute(query)

        # Eredmények dic-ek listájába
        records = [dict(row) for row in cursor.fetchall()]
        cursor.close()
        conn.close()

        return records

    except Exception as e:
        print("Hiba történt:", e)
        return None

records = get_userid_sql()
records

[{'order_id': 236086,
  'user_id': 742,
  'order_number': 3,
  'order_dow': 0,
  'order_hour_of_day': 14,
  'days_since_prior_order': 17.0,
  'product_id': 79,
  'add_to_cart_order': 1,
  'reordered': 1,
  'department_id': 1,
  'department': 'frozen',
  'product_name': 'frozen pizza'},
 {'order_id': 236086,
  'user_id': 742,
  'order_number': 3,
  'order_dow': 0,
  'order_hour_of_day': 14,
  'days_since_prior_order': 17.0,
  'product_id': 116,
  'add_to_cart_order': 2,
  'reordered': 0,
  'department_id': 1,
  'department': 'frozen',
  'product_name': 'frozen produce'},
 {'order_id': 236086,
  'user_id': 742,
  'order_number': 3,
  'order_dow': 0,
  'order_hour_of_day': 14,
  'days_since_prior_order': 17.0,
  'product_id': 91,
  'add_to_cart_order': 3,
  'reordered': 0,
  'department_id': 16,
  'department': 'dairy eggs',
  'product_name': 'soy lactosefree'},
 {'order_id': 236086,
  'user_id': 742,
  'order_number': 3,
  'order_dow': 0,
  'order_hour_of_day': 14,
  'days_since_prior_or

## Számold ki Pythonban, hogy hány user rendelt legalább 10-szer.

In [11]:
import psycopg2
import yaml
import pandas as pd

def get_10orders_sql():
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # Lekérdezés
        query = """
        with cte as (
            select distinct user_id, count(order_id) cnt
            from consumer_behavior
            group by user_id
            having count(order_id) > 9
            order by cnt
        ) select count(cnt) from cte;
        """
        df = pd.read_sql_query(query, conn)
        conn.close()

        return df

    except Exception as e:
        print("Hiba történt:", e)
        return None

records = get_10orders_sql()
records

/tmp/ipykernel_3177636/1965786556.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,count
0,65842


## Írj kódot, ami kiszedi azokat az order_id-ket, ahol több mint 15 termék van.

In [12]:
import psycopg2
import yaml
import pandas as pd

def get_morethan15_sql():
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # Lekérdezés
        query = """
        SELECT order_id, COUNT(product_id) cnt
        FROM consumer_behavior
        GROUP BY order_id
        HAVING COUNT(product_id) > 15
        ORDER BY cnt DESC
        LIMIT 10;
        """
        df = pd.read_sql_query(query, conn)
        conn.close()

        return df

    except Exception as e:
        print("Hiba történt:", e)
        return None

records = get_morethan15_sql()
records

/tmp/ipykernel_3177636/4258713010.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,order_id,cnt
0,790903,137
1,2621625,109
2,416346,100
3,1031566,95
4,1930316,93
5,2409933,93
6,45973,92
7,2926893,92
8,936852,87
9,70299,83


# 📊 5. SZEKCIÓ – Pandas + SQL + elemzés

Pandas-szal töltsd be az SQL táblát DataFrame-be (pd.read_sql()).

## department szerint számold meg a rendeléseket.

## számold meg, ki hány terméket rendelt.

## átlagosan hány termék van egy order_id-ben?

## Készíts pivot táblát: order_dow × department, érték = db szám.

In [1]:
import psycopg2
import pandas as pd
import yaml

def pivot_order_dow_department():
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        # Mivel a fájlban 'database' kulcs alatt vannak az adatok
        db_config = config['database']

        # Kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        # Először lekérdezzük az összes distinct department-et
        query = "SELECT order_dow, department FROM consumer_behavior LIMIT 10000;"
        df = pd.read_sql_query(query, conn)
        conn.close()

        # Pivot tábla készítése
        pivot_df = df.pivot_table(
            index = 'order_dow',
            columns = 'department',
            aggfunc = 'size', # COUNT
            fill_value = 0 # ha nincs érték, legyen 0
        )

        # Opcionálisan visszaállítjuk az oszlop- és index neveket a szebb megjelenéshez
        pivot_df = pivot_df.reset_index().rename_axis(None, axis=1)

        return pivot_df

    except Exception as e:
        print("Hiba történt:", e)
        return None

pivot_table = pivot_order_dow_department()
pivot_table

/tmp/ipykernel_3183324/1223278613.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,order_dow,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,0,2,18,78,109,28,3,64,321,56,...,37,21,46,8,1,97,20,7,638,145
1,1,8,10,65,179,51,1,64,284,65,...,30,29,32,5,2,107,18,7,551,159
2,2,2,21,51,98,52,1,46,214,44,...,38,9,25,2,1,79,19,11,383,101
3,3,8,6,36,97,28,1,30,169,41,...,19,1,16,2,1,60,8,2,299,76
4,4,12,6,50,108,39,0,40,215,50,...,30,11,31,2,2,65,13,3,299,136
5,5,7,11,53,158,35,4,45,272,38,...,44,10,30,1,1,80,22,2,434,126
6,6,5,17,39,85,17,2,48,204,57,...,40,16,29,1,3,80,9,6,410,92


## Szűrd ki azokat az order_id-ket, ahol csak 1 darab termék van.

## Számold ki, melyik termék a leggyakoribb az új (reordered=0) rendelésekben.

## Csoportosíts: hány rendelés történt délután (12–18 óra között)?

# ⚙️ 6. SZEKCIÓ – Függvényírás gyakorlása

In [ ]:
# Kernel dies:
import pandas as pd
dfs = []
for chunk in pd.read_csv("ECommerce_consumer_behaviour.csv", encoding='utf-8', sep=',', chunksize=50000):
    dfs.append(chunk)
df = pd.concat(dfs)

In [ ]:
# Kernel dies:
import pandas as pd
df =  pd.read_csv("ECommerce_consumer_behaviour.csv", encoding='utf-8', sep=',', low_memory=False, memory_map=True)

In [ ]:
# Kernel dies
import pandas as pd
dtypes = {
    "order_id": "int32",
    "user_id": "int32",
    "order_number": "int16",
    "order_dow": "int8",
    "order_hour_of_day": "int8",
    "days_since_prior_order": "float32",
    "product_id": "int32",
    "add_to_cart_order": "int16",
    "reordered": "int8",
    "department_id": "int8",
    "department": "category",
    "product_name": "category"
}

df = pd.read_csv(
    "ECommerce_consumer_behaviour.csv",
    sep=",",
    dtype=dtypes,
    low_memory=False,
    encoding="utf-8"
)

In [1]:
import pandas as pd

df = pd.read_csv("ECommerce_consumer_behaviour.csv", encoding='utf-8', sep=',', nrows=500000)
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


## Írj függvényt: get_top_products_by_department(department_name, limit) → SQL + Pandas visszaad DataFrame-et.

### psycopg2

In [2]:
import psycopg2
import yaml
import pandas as pd

def get_top_products_by_department(department_name, limit):
    try:
        # yaml beolvasás
        yaml_path="/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db_config = config['database']

        # psycopg2-kapcsolódás PostgreSQL-hez
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # SQL Lekérdezés
        query = """
        WITH cte AS (
            SELECT
                department
                ,product_name
                ,COUNT(*) cnt
                ,ROW_NUMBER() OVER(partition by department order by COUNT(*) DESC) rn
            FROM consumer_behavior
            GROUP BY department, product_name)
        SELECT
            department
            ,product_name
            ,cnt
        FROM cte
        WHERE 
            department = %s 
            AND 
            rn <= %s
        ORDER BY department, cnt DESC;
        """

        # Pandas Dataframe-be való olvasás
        df = pd.read_sql_query(query, conn, params=(department_name, limit))
        conn.close()

        return df

    except Exception as e:
        print("Hiba történt:", e)
        return None

df_top = get_top_products_by_department("pantry", 5)
df_top

/tmp/ipykernel_3222858/3075449742.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(department_name, limit))


,department,product_name,cnt
0,pantry,baking ingredients,20137
1,pantry,spreads,18149
2,pantry,oils vinegars,15213
3,pantry,condiments,13760
4,pantry,spices seasonings,13067


### SQLAlchemy

In [4]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def get_top_products_by_department(department_name, limit):
    try:
        # 1. YAML beolvasása
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)

        db = config['database']

        # 2. SQLAlchemy connection string
        conn_str = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@"
            f"{db['host']}:{db['port']}/{db['dbname']}"
        )

        # 3. Engine létrehozása
        engine = create_engine(conn_str)

        # 4. SQL lekérdezés
        sql = text("""
            WITH cte AS (
                SELECT
                    department,
                    product_name,
                    COUNT(*) AS cnt,
                    ROW_NUMBER() OVER (
                        PARTITION BY department
                        ORDER BY COUNT(*) DESC
                    ) AS rn
                FROM consumer_behavior
                GROUP BY department, product_name
            )
            SELECT
                department,
                product_name,
                cnt
            FROM cte
            WHERE 
              department = :dept
              AND 
              rn <= :lim
            ORDER BY cnt DESC;
        """)

        # 5. Eredmény betöltése DataFrame-be
        df = pd.read_sql(sql, engine, params={"dept": department_name, "lim": limit})

        return df

    except Exception as e:
        print("Hiba történt:", e)
        return None

df_top = get_top_products_by_department("dairy eggs", 5)
df_top

,department,product_name,cnt
0,dairy eggs,yogurt,90751
1,dairy eggs,packaged cheese,61502
2,dairy eggs,milk,55150
3,dairy eggs,soy lactosefree,39389
4,dairy eggs,eggs,27986


## Írj függvényt, amely megnézi egy user törzsvásárló-e (legalább 5-ször ugyanazt a terméket rendelte).

### psycopg2

In [9]:
import psycopg2
import pandas as pd
import yaml

def regular_customer():
    try:
        # Authentication
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db_config = config['database']

        # Connecting to database
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )

        cursor = conn.cursor()

        # SQL query
        query = """
        WITH cte AS (
            SELECT
                user_id
                ,product_id
                ,COUNT(*) cnt
                ,CASE WHEN COUNT(*) >= 5 THEN 1 ELSE 0 END AS regular_customer
            FROM consumer_behavior
            GROUP BY user_id, product_id)
        SELECT
            user_id
            ,CASE WHEN SUM(regular_customer) = 0 THEN 'Not regular' ELSE 'Regular' END AS reg
        FROM cte 
        GROUP BY user_id
        ORDER BY user_id;
        """

        # Creating Pandas dataframe
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df

    except Exception as e:
        print("Hiba történt", e)
        return None

df_regular_customer = regular_customer()
df_regular_customer

/tmp/ipykernel_3222858/772107008.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,user_id,reg
0,2,Not regular
1,3,Not regular
2,7,Regular
3,10,Regular
4,11,Not regular
...,...,...
105268,206204,Not regular
105269,206205,Not regular
105270,206206,Not regular
105271,206208,Regular


### SQLAlchemy

In [11]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def regular_customer_alch():
    try:
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db = config['database']

        # SQLAlchemy
        conn_str = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}"
        )

        # Create engine
        engine = create_engine(conn_str)

        # SQL query
        sql = text("""
            WITH cte AS (
                SELECT
                    user_id
                    ,product_id
                    ,COUNT(*) cnt
                    ,CASE WHEN COUNT(*) >= 5 THEN 1 ELSE 0 END AS regular_customer
                FROM consumer_behavior
                GROUP BY user_id, product_id)
            SELECT
                user_id
                ,CASE WHEN SUM(regular_customer) = 0 THEN 'Not regular' ELSE 'Regular' END AS reg
            FROM cte 
            GROUP BY user_id
            ORDER BY user_id;
        """)
    
        # Loading into dataframe
        df = pd.read_sql(sql, engine)
        return df
        
    except Exception as e:
        print("Hiba:", e)
        return None

df_reg_cust = regular_customer_alch()
df_reg_cust

,user_id,reg
0,2,Not regular
1,3,Not regular
2,7,Regular
3,10,Regular
4,11,Not regular
...,...,...
105268,206204,Not regular
105269,206205,Not regular
105270,206206,Not regular
105271,206208,Regular


## Függvény: számolja ki a reorder arányt egy adott departmentre.

### psycopg2

In [5]:
import pandas as pd
import psycopg2
import yaml

def get_reorder_rate_by_department_psy(department_name):
    try:
        # yaml
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db_config = config['database']
    
        # Connecting to database via psycopg2
        conn = psycopg2.connect(
            host = db_config['host'],
            port = db_config['port'],
            dbname = db_config['dbname'],
            user = db_config['user'],
            password = db_config['password']
        )
    
        cursor = conn.cursor()
    
        # SQL query
        query = """
        SELECT
          COUNT(*) FILTER (WHERE reordered = 1) AS reordered_count,
          COUNT(*) AS total_count,
          (COUNT(*) FILTER (WHERE reordered = 1))::float / NULLIF(COUNT(*),0) * 100 AS reorder_rate
        FROM consumer_behavior
        WHERE department = %s;
        """

        # Loading result into dataframe
        df = pd.read_sql_query(query, conn, params=(department_name,))
        return df
        
    except Exception as e:
        print("Hiba:", e)
        return None

df_dep_rate_psy = get_reorder_rate_by_department_psy("dairy eggs")
df_dep_rate_psy    

/tmp/ipykernel_3248017/105194255.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(department_name,))


,reordered_count,total_count,reorder_rate
0,225798,336915,67.019278


### SQLAlchemy

In [3]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def get_reorder_rate_by_department(department_name):
    try:
        #yaml
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db = config['database']
                
        # Connecting to database via SQLAlchemy
        conn_str = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}"
        )

        # Creating engine
        engine = create_engine(conn_str)

        # SQL query
        sql = text("""
        SELECT
          COUNT(*) FILTER (WHERE reordered = 1) AS reordered_count,
          COUNT(*) AS total_count,
          (COUNT(*) FILTER (WHERE reordered = 1))::float / NULLIF(COUNT(*),0) * 100 AS reorder_rate
        FROM consumer_behavior
        WHERE department = :dept;
        """)

        # Loading result into dataframe
        df = pd.read_sql(sql, engine, params = {"dept": department_name})
        return df
        
    except Exception as e:
        print("Hiba:", e)
        return None

df_dep_rate = get_reorder_rate_by_department("produce")
df_dep_rate

,reordered_count,total_count,reorder_rate
0,382425,588996,64.928285


## Függvény: Melyik órában a legnagyobb az esélye annak, hogy egy rendelés „reordered” lesz?

In [4]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def which_hour_reorder():
    try:
        # yaml
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db = config['database']
        
        # Database
        conn_str = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}"
        )
        
        # engine
        engine = create_engine(conn_str)
        
        # query
        sql = text("""
        WITH cte AS (
            SELECT
                order_hour_of_day
                ,COUNT(*) total_order
                ,COUNT(*) FILTER (WHERE reordered = 1) reordered_order
                ,ROUND( 
                    ((COUNT(*) FILTER (WHERE reordered = 1))::float 
                      / NULLIF(COUNT(*),0) * 100)::numeric, 2) AS reorder_rate
            FROM consumer_behavior
            GROUP BY order_hour_of_day)
        SELECT 
            order_hour_of_day
            ,reorder_rate
        FROM cte
        ORDER BY reorder_rate DESC;
        """)
        
        # dataframe
        df = pd.read_sql(sql, engine)
        return df

    except Exception as e:
        print("Hiba:", e)
        return None

df_hour = which_hour_reorder()
df_hour.head(1)

,order_hour_of_day,reorder_rate
0,7,64.53


## Függvény: TOP 3 termék minden departmentben (nested groupby).

In [11]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def top_3_product(department):
    try:
        # yaml
        yaml_path = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
        db = config['database']
        
        # sqlalchemy
        conn_str = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}"
        )
        
        # engine
        engine = create_engine(conn_str)

        # query
        query = text("""
        WITH cte AS (
            SELECT
                department
                ,product_name
                ,COUNT(*) cnt
            FROM consumer_behavior
            GROUP BY department, product_name),
        cte2 AS
            (SELECT 
                *
                ,ROW_NUMBER() OVER(partition by department order by cnt DESC) rn
            FROM cte)
        SELECT
            department
            ,product_name
            ,rn
        FROM cte2
        WHERE rn <= 3 AND department = :dept;
        """)
        
        # dataframe
        df = pd.read_sql(query, engine, params= {"dept": department})
        return df

    except Exception as e:
        print("Hiba:", e)
        return None

df_top3 = top_3_product('alcohol')
df_top3

,department,product_name,rn
0,alcohol,beers coolers,1
1,alcohol,red wines,2
2,alcohol,white wines,3


## Függvény: vizsgálja meg, hogy egy order_id tartalmaz-e duplikált product_id-t, és adjon vissza True/False.

In [4]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

def has_duplicate(order_id):
    try:
        # yaml
        auth = "/home/gdaniel1979/auth/ecommerce.yaml"
        with open(auth, 'r') as f:
            config = yaml.safe_load(f)
        db = config['database']
    
        # sqlalchemy
        conn = (
            f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}"
        )
    
        # engine
        engine = create_engine(conn)
    
        # query
        query = text("""
        SELECT 
            order_id,
            CASE 
                WHEN COUNT(product_id) > COUNT(DISTINCT product_id) 
                THEN true 
                ELSE false 
            END AS has_duplicate
        FROM consumer_behavior
        GROUP BY order_id
        HAVING order_id = :orderid;
        """)
    
        # df
        df = pd.read_sql(query, engine, params={"orderid": order_id})

        if df.empty:
            print(f"Nincs ilyen order_id: {order_id}")
            return None
        return df

    except Exception as e:
        print("Hiba:", e)
        return None

df_has_duplicates = has_duplicate(10)
df_has_duplicates

,order_id,has_duplicate
0,10,True


🎯 8. SZEKCIÓ – Mini adatkutató projektek

Készíts riportot: "TOP 5 legaktívabb user, részletes bontással".

Elemezd, hogy melyik napszakban a legmagasabb a reorder arány (reggel-délelőtt-délután-este kategóriák).

In [ ]:
Fedezd fel, hogy egyes departmentekben van-e szezonális minta order_dow szerint.

Készíts egy "buyer profile" elemzést: tipikus produce-vásárlók milyen más departmentből vásárolnak még?

Generálj automatikusan egy szöveges jelentést Pythonban, pl.:

"A legtöbb rendelés vasárnap történik (23%). A dairy eggs kategóriában a reorder arány 61%, ami magasabb az átlagos 48%-nál. A legaktívabb időszak délután 15:00 – 17:00." (→ print-el jelenjen meg)